In [ ]:
!pip install fastapi uvicorn["standard"]
!pip install nest-asyncio
# !pip install transformers
# !pip install https://github.com/huggingface/transformers/archive/nllb.zip -q
!pip install easygoogletranslate
!pip install pyngrok
!pip install selenium

In [ ]:
from fastapi import FastAPI
from pydantic import BaseModel
import requests
from easygoogletranslate import EasyGoogleTranslate
from pyngrok import ngrok
import nest_asyncio
from fastapi.middleware.cors import CORSMiddleware
import uvicorn
import PyPDF2
import langchain
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
import json
from fastapi import FastAPI, File, UploadFile
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import json
from io import BytesIO
import openai
import torch
import torch.nn.functional as F


app = FastAPI()

port=8000
app.add_middleware(
CORSMiddleware,
allow_origins=["*"],
allow_credentials=True,
allow_methods=["*"],
allow_headers=["*"],
)

lang_json={
  "languages": [
    {"name": "Afrikaans", "code": "af"},
    {"name": "Albanian", "code": "sq"},
    {"name": "Amharic", "code": "am"},
    {"name": "Arabic", "code": "ar"},
    {"name": "Armenian", "code": "hy"},
    {"name": "Azerbaijani", "code": "az"},
    {"name": "Basque", "code": "eu"},
    {"name": "Belarusian", "code": "be"},
    {"name": "Bengali", "code": "bn"},
    {"name": "Bosnian", "code": "bs"},
    {"name": "Bulgarian", "code": "bg"},
    {"name": "Catalan", "code": "ca"},
    {"name": "Cebuano", "code": "ceb"},
    {"name": "Chinese (Simplified)", "code": "zh-CN"},
    {"name": "Chinese (Traditional)", "code": "zh-TW"},
    {"name": "Corsican", "code": "co"},
    {"name": "Croatian", "code": "hr"},
    {"name": "Czech", "code": "cs"},
    {"name": "Danish", "code": "da"},
    {"name": "Dutch", "code": "nl"},
    {"name": "English", "code": "en"},
    {"name": "Esperanto", "code": "eo"},
    {"name": "Estonian", "code": "et"},
    {"name": "Finnish", "code": "fi"},
    {"name": "French", "code": "fr"},
    {"name": "Frisian", "code": "fy"},
    {"name": "Galician", "code": "gl"},
    {"name": "Georgian", "code": "ka"},
    {"name": "German", "code": "de"},
    {"name": "Greek", "code": "el"},
    {"name": "Gujarati", "code": "gu"},
    {"name": "Haitian Creole", "code": "ht"},
    {"name": "Hausa", "code": "ha"},
    {"name": "Hawaiian", "code": "haw"},
    {"name": "Hebrew", "code": "he"},
    {"name": "Hindi", "code": "hi"},
    {"name": "Hmong", "code": "hmn"},
    {"name": "Hungarian", "code": "hu"},
    {"name": "Icelandic", "code": "is"},
    {"name": "Igbo", "code": "ig"},
    {"name": "Indonesian", "code": "id"},
    {"name": "Irish", "code": "ga"},
    {"name": "Italian", "code": "it"},
    {"name": "Japanese", "code": "ja"},
    {"name": "Javanese", "code": "jw"},
    {"name": "Kannada", "code": "kn"},
    {"name": "Kazakh", "code": "kk"},
    {"name": "Khmer", "code": "km"},
    {"name": "Korean", "code": "ko"},
    {"name": "Kurdish", "code": "ku"},
    {"name": "Kyrgyz", "code": "ky"},
    {"name": "Lao", "code": "lo"},
    {"name": "Latin", "code": "la"},
    {"name": "Latvian", "code": "lv"},
    {"name": "Lithuanian", "code": "lt"},
    {"name": "Luxembourgish", "code": "lb"},
    {"name": "Macedonian", "code": "mk"},
    {"name": "Malagasy", "code": "mg"},
    {"name": "Malay", "code": "ms"},
    {"name": "Malayalam", "code": "ml"},
    {"name": "Maltese", "code": "mt"},
    {"name": "Maori", "code": "mi"},
    {"name": "Marathi", "code": "mr"},
    {"name": "Mongolian", "code": "mn"},
    {"name": "Myanmar (Burmese)", "code": "my"},
    {"name": "Nepali", "code": "ne"},
    {"name": "Norwegian", "code": "no"},
    {"name": "Nyanja (Chichewa)", "code": "ny"},
    {"name": "Pashto", "code": "ps"},
    {"name": "Persian", "code": "fa"},
    {"name": "Polish", "code": "pl"},
    {"name": "Portuguese (Portugal, Brazil)", "code": "pt"},
    {"name": "Punjabi", "code": "pa"},
    {"name": "Romanian", "code": "ro"},
    {"name": "Russian", "code": "ru"},
    {"name": "Samoan", "code": "sm"},
    {"name": "Scots Gaelic", "code": "gd"},
    {"name": "Serbian", "code": "sr"},
    {"name": "Sesotho", "code": "st"},
    {"name": "Shona", "code": "sn"},
    {"name": "Sindhi", "code": "sd"},
    {"name": "Sinhala (Sinhalese)", "code": "si"},
    {"name": "Slovak", "code": "sk"},
    {"name": "Slovenian", "code": "sl"},
    {"name": "Somali", "code": "so"},
    {"name": "Spanish", "code": "es"},
    {"name": "Sundanese", "code": "su"},
    {"name": "Swahili", "code": "sw"},
    {"name": "Swedish", "code": "sv"},
    {"name": "Tagalog (Filipino)", "code": "tl"},
    {"name": "Tajik", "code": "tg"},
    {"name": "Tamil", "code": "ta"},
    {"name": "Telugu", "code": "te"},
    {"name": "Thai", "code": "th"},
    {"name": "Turkish", "code": "tr"},
    {"name": "Ukrainian", "code": "uk"},
    {"name": "Urdu", "code": "ur"},
    {"name": "Uzbek", "code": "uz"},
    {"name": "Vietnamese", "code": "vi"},
    {"name": "Welsh", "code": "cy"},
    {"name": "Xhosa", "code": "xh"},
    {"name": "Yiddish", "code": "yi"},
    {"name": "Yoruba", "code": "yo"},
    {"name": "Zulu", "code": "zu"}
  ]
}

# with open('env.txt','r') as f:
# document = './resumes/kreena.pdf'


def parse_output(project_string):
    lines = project_string.strip().split('\n')
    project_info = {}
    for line in lines:
        key, value = map(str.strip, line.split(':', 1))
        if key == "Project Tags":
            value = [tag.strip() for tag in value.split(',')]
        project_info[key] = value

    # Convert the information to JSON
    json_output = json.dumps(project_info, indent=4)

    return json_output

def scrape_project(project_link):
    options = webdriver.FirefoxOptions()
    options.add_argument("--headless")
    # Initialize the webdriver
    driver = webdriver.Firefox(options=options)
    driver.get(project_link)
    # time.sleep(20)
    # print(driver.find_element(By.CSS_SELECTOR, 'h4'))
    elem = driver.find_elements(By.CLASS_NAME, 'work-key-word-btn')
    tags = []
    for e in elem:
        tags.append(e.text)

    name = driver.find_element(By.CLASS_NAME, "timeline-title").text
    desc = driver.find_element(By.CLASS_NAME, 'user-timeline-descriptions').text

    driver.close()
    project_info = {}
    project_info['Title'] = name
    project_info["Project Tags"] = tags
    project_info['Description'] = desc

    return json.dumps(project_info, indent=4)


class TransText(BaseModel):
    text: str
    to:str

class ScrapeProfileRequest(BaseModel):
  link:str

class CompareRequest(BaseModel):
    user_tags:str
    job_tags:str

class RecommendRequest(BaseModel):
    domain:str
        
@app.get("/")
async def root():
    return {"message": "Turned On"}

# @app.post("/detect_language")

@app.post("/translate_language")
async def translate(request: TransText):
  text = request.text
  to = request.to
  for lang in lang_json['languages']:
    if lang['name'] == request.to:
        english = lang

  code = english['code']
  translator = EasyGoogleTranslate(
    source_language='en',
    target_language=code,
    timeout=10
)
  result = translator.translate(request.text)
  return result



@app.post("/resume_parse")
async def parse_resume(file: UploadFile = File(...)):
    file_bytes = await file.read()
    print("aya")
    # Create a BytesIO object 
    pdf_stream = BytesIO(file_bytes)
    
    pdf_reader = PyPDF2.PdfReader(pdf_stream)
    text = pdf_reader.pages[0].extract_text()
    template1 = f"""
       % INSTRUCTIONS
       Format only the following resume projects in the given format. (Exclude other headers like education and extracurricular).

       % INPUT
       Resume : {text}

       % Output
        Title: [title]
        Project Tags: [list of all tools and techniques used]
        Description: [Description]
       """

    prompt1 = PromptTemplate.from_template(template=template1)
    print("before openai")
    llm = ChatOpenAI(openai_api_key=api_key, max_tokens=2048, model_name='gpt-3.5-turbo')
    # chain1 = LLMChain(llm=llm, prompt=prompt1)
    out = llm.predict(template1)
    print("after openai")
    p = out.split('\n\n')

    projects = []

    for l in p:
        projects.append(parse_output(l))
    print("jaraha hai")
    return projects

@app.post("/scrape_profile")
async def scrape_profile(request: ScrapeProfileRequest):
  link = request.link
  options = webdriver.FirefoxOptions()
  options.add_argument("--headless")

    # Initialize the webdriver
  driver = webdriver.Firefox(options=options)

  if link[-1] != '/':
      link = link + '/'

    # Navigate to a website
  driver.get(link)  # Replace with the URL of the website you want to visit

  time.sleep(2)

  elem = driver.find_elements(By.CSS_SELECTOR, "a")

  projects = []
  social = []

  for e in elem:
      x = e.get_attribute("href")
      if link in x and not ('contribution' in x or 'timeline' in x or 'collaborations' in x):
          projects.append(x)
      for sm in ['twitter.com', 'facebook.com', 'instagram.com', 'linkedin.com', 'youtube.com', 'behance.com', 'figma.com', 'bit.ly']:
          if sm in x:
              social.append(x)

  driver.close()
  scraped_project=[]
  for p in projects:
    scraped_project.append(scrape_project(p))
  return scraped_project, social

@app.post("/compare")
async def compare(request:CompareRequest):
    usr_tags_str, job_tags = request.user_tags,request.job_tags
#     usr_tags_str = " ".join(usr_tags)
    usr_tags = usr_tags_str.split(',')
    #     print(usr_tags_str, job_tags_str)
    #     job_tags_str = job_tags_str[1:-1].replace('"','')
    job_tags = json.loads(job_tags)
    print(usr_tags, job_tags)
    #     job_tags = json.load(job_tags)
    #     job_tags = job_tags.values()
    job_tags_str = " ".join(job_tags)
    usr_emb = openai.Embedding.create(model='text-embedding-ada-002', input=usr_tags_str)["data"]
    # usr_emb = torch.tensor([e['embedding'] for e in usr_emb])
    job_emb = openai.Embedding.create(model='text-embedding-ada-002', input=job_tags_str)["data"]
    # job_emb = torch.tensor([e['embedding'] for e in job_emb])

    usr_emb = torch.tensor(usr_emb[0]['embedding'])
    job_emb = torch.tensor(job_emb[0]['embedding'])

    # usr_emb = usr_emb.mean(axis=0)
    # job_emb = job_emb.mean(axis=0)

    current_sim = F.cosine_similarity(usr_emb, job_emb, dim=0)
    print(current_sim)

    tag_priority = []

    for tag in job_tags:
        if tag not in usr_tags:
            new_usr_tags = usr_tags + [tag]
            new_usr_tags_str = " ".join(new_usr_tags)
            new_usr_emb = openai.Embedding.create(model='text-embedding-ada-002', input=new_usr_tags_str)["data"]

            new_usr_emb = torch.tensor(new_usr_emb[0]['embedding'])
            tag_sim = F.cosine_similarity(new_usr_emb, job_emb, dim=0)

            tag_priority.append((tag, tag_sim - current_sim))

    tag_priority.sort(key=lambda x:x[1], reverse=True)
    print(tag_priority)
    #     return tag_priority
    revised_list=[]
    for tag in tag_priority:
        revised_list.append(tag[0])
    return {"tags":revised_list, "sim":current_sim.item()}

@app.post("/recommend")
async def recommend_project(request:RecommendRequest):
    domain = request.domain
    template = f"""
           % INSTRUCTIONS
           Suggest 2 project recommendations to build skill in the required domain below. Explain the idea in brief and touch on how it is relevant to current market demand and how it boosts the resume.
    
           % INPUT
           Domain : {domain}

           % Output
            Project Idea1 : [idea1]
            Project Idea2 : [idea2]
           """

    llm = ChatOpenAI(openai_api_key=api_key, max_tokens=2048, model_name='gpt-3.5-turbo')
    out = llm.predict(template)
    return out

@app.post("/resources")
async def resources(request:RecommendRequest):
    domain = request.domain
    options = webdriver.FirefoxOptions()
    options.add_argument("--headless")

    # Initialize the webdriver
    driver = webdriver.Firefox(options=options)
    driver.get(f"https://www.udemy.com/courses/search/?src=ukw&q={domain}")
    time.sleep(4)
    elem = driver.find_elements(By.CLASS_NAME, 'course-card-title-module--title--2C6ac')[:3]
    courses = []
    for e in elem:
        courses.append({"course": e.text, "link": e.find_element(By.CSS_SELECTOR, 'a').get_attribute('href')})
    return json.dumps(courses, indent=4)


if __name__=="__main__":
    ngrok_tunnel = ngrok.connect(port)
    print('Public URL:', ngrok_tunnel.public_url)
    nest_asyncio.apply()
    uvicorn.run(app,port=port)

t=2023-10-29T10:43:22+0530 lvl=warn msg="ngrok config file found at legacy location, move to XDG location" xdg_path=C:\\Users\\maitr\\AppData\\Local/ngrok/ngrok.yml legacy_path=C:\\Users\\maitr\\.ngrok2\\ngrok.yml
INFO:     Started server process [35020]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


Public URL: https://85fc-2409-40c0-47-d56c-9590-8abf-c2a4-649c.ngrok-free.app
